In [1]:
import pandas as pd
import numpy as np
import scipy
import pickle

from sklearn.model_selection import train_test_split

news_data = pd.read_csv("/content/drive/My Drive/Data/preproccessed_data.csv")

news_data.index.name = 'index'
news_data

,title,text,label
index,,,
0,muslims busted stole millions gov ’ benefits,print pay back money plus interest entire fami...,1
1,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1
2,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1
3,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1
4,fantastic trumps 7 point plan reform healthcar...,email healthcare reform make america great sin...,1
...,...,...,...
29449,process analytical instruments market – techna...,technavio published new report global process ...,0
29450,travel deals get 1200 air credit two apt cookt...,apt offering savings new cape york outback wil...,0
29451,taiwanese recyclers belief waste simply mispla...,taipei taiwan sept 8 2015 prnewswire recent ye...,0


In [2]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from gensim.test.utils import get_tmpfile

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=1000, window=2, min_count=1, workers=4)

fname = get_tmpfile("my_doc2vec_model")


model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


def doc2vec_fun(x):
    L = []
    punc = ',"»-'
    for elem in x.split(' '):
        if elem not in punc:
          L.append(elem)
    vector = model.infer_vector(L)
    return vector
news_data['doc2vec_title']=news_data['title'].map(lambda x:doc2vec_fun(str(x)))
news_data['doc2vec_text']=news_data['text'].map(lambda x:doc2vec_fun(str(x)))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
# this is only the text data which has been converted to a data frame.
# each row is one article and each column is an element of the vector. 

text_vectorised_data = pd.DataFrame(news_data['doc2vec_text'].array, columns = range(1000))
title_vectorised_data = pd.DataFrame(news_data['doc2vec_title'].array, columns = range(1000))

# once we have vectorised the entire datset then split into train and test! 
text_vec_train, text_vec_test, label_train, label_test = train_test_split(text_vectorised_data, news_data['label'], test_size=0.33, 
                                                        random_state=88)

In [4]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
# trying different SVM kernels
from sklearn import metrics

def svm_classifier(X, y, Xtest, ytest):
    
    models = [SVC(kernel = 'linear'),
             SVC(kernel = 'poly'),
             SVC(kernel = 'rbf'),
             SVC(kernel = 'sigmoid')]
    
    titles = ['SVC with linear kernel',
            'SVC with polynomial kernel',
             'SVC with rbf kernel',
             'SVC with sigmoid kernel']


    
    # prepare the cross-validation procedure
    cv = KFold(n_splits=4, random_state=1, shuffle=True)
    
    for title, model in zip(titles, models):
        print('\n',title, ': with doc2vec')
        model.fit(X, y)
        y_pred = model.predict(Xtest)
        print(y_pred.sum())
        print("Accuracy:",metrics.accuracy_score(ytest, y_pred))
        # Model Precision: what percentage of positive tuples are labeled as such?
        print("Precision:",metrics.precision_score(ytest, y_pred))
        # Model Recall: what percentage of positive tuples are labelled as such?
        print("Recall:",metrics.recall_score(ytest, y_pred))
            
    return

svm_classifier(text_vec_train, label_train, text_vec_test, label_test)


 SVC with linear kernel : with doc2vec
0
Accuracy: 0.5700617283950618
Precision: 0.0
Recall: 0.0

 SVC with polynomial kernel : with doc2vec


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# trying different SVM kernels

def logreg_classifier(X, y):
    

    models = [LogisticRegression(solver = 'newton-cg'),
             LogisticRegression(solver = 'lbfgs'),
             LogisticRegression(solver = 'liblinear'),
             LogisticRegression(solver = 'sag'),
             LogisticRegression(solver = 'saga')]
    
    titles = ['Logistic Regression with newton-cg',
            'Logistic Regression with lbfgs',
             'Logistic Regression with liblinear',
             'Logistic Regression with sag',
             'Logistic Regression with saga']


    
    # prepare the cross-validation procedure
    cv = KFold(n_splits=4, random_state=1, shuffle=True)
    
    for title, model in zip(titles, models):
        print('\n',title, ': doc2vec')
        model = model
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
        print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
            
    return

logreg_classifier(text_vectorised_data, news_data['label'])